# White-box identification with state-space model

Last update: 17-10-2023

---

The goal of this project is to identify dynamics in a grey-box model of a thermal setup (see system description below), based on temperature measurements at specific locations on the system. Parameters include conductance and linear convection.

In [ ]:
import Pkg
Pkg.activate("../..")
Pkg.instantiate();

## System description

A schematic depiction of the setup is shown in the figure below. 

<p><center><img src='../../system/system-photo.png'/></center></p>

In short, the setup consists of 3 metal blocks which have been lined up, with resistive nylon pads interposed. The temperature can be measured using thermistors at arbitrary places on the setup; for simplicity we assume that we measure the temperature at a single spot on each block, which we call $\tau_1$, $\tau_2$, and $\tau_3$. The temperatures will evolve due to a number of different factors; we will only consider the influence of conduction, convection, radiation, and the user controlled input heat (band heaters).

By assuming that conduction within blocks is so fast that there are no temperature differences within a block, we may model the system using a [lumped-element model](https://en.wikipedia.org/wiki/Lumped-element_model), governed by the following system of ODEs:

$$\frac{d}{dt}\begin{pmatrix} m_1 c_{p, 1} \tau_1 \\ m_2 c_{p, 2} \tau_2 \\ m_3 c_{p, 3} \tau_3 \end{pmatrix} = 
\underbrace{\begin{pmatrix} -k_{12} & k_{12} & 0 \\ k_{12} & -(k_{12} + k_{23}) & k_{23} \\ 0 & k_{23} & -k_{23} \end{pmatrix} \begin{pmatrix} \tau_1 \\ \tau_2 \\ \tau_3 \end{pmatrix}}_{\textrm{conduction}} + \underbrace{\begin{pmatrix} h(\tau_1, \tau_a, 1, t) \\ h(\tau_2, \tau_a, 2, t) \\ h(\tau_3, \tau_a, 3, t) \end{pmatrix}}_{\textrm{convection}} + \underbrace{\sigma \begin{pmatrix} a_1 \varepsilon_1 (\tau_a^4 - \tau_1^4) \\ a_2 \varepsilon_2 (\tau_a^4 - \tau_2^4) \\ a_3 \varepsilon_3 (\tau_a^4 - \tau_3^4) \end{pmatrix}}_{\textrm{radiation}} + \underbrace{\begin{pmatrix} u_1 \\ u_2 \\ u_3 \end{pmatrix}}_{\textrm{input}}.$$

Convection is notoriously hard to model. A coarse approximation would be Newton's law of cooling (Clercx, 2015; Eq. 8.17), which states that convection is linear in the difference between the temperature of the block and the ambient temperature: $h_a (\tau_a - \tau_i)$. With this linear term, we can take steps similar to the identification of the oscillator in Rogers \& Friis (2022), describing the nonlinear function as the combination of a linear term and a "nonlinear remainder":

$$\underbrace{h(\tau_i, \tau_a, i, t)}_{\text{total convection}} = \underbrace{h_a (\tau_a - \tau_i)}_{\text{linear cooling law}} + \underbrace{r(\tau_i, \tau_a, i, t)}_{\text{nonlinear remainder}} \, ,$$

for some constant $h_a$. Furthermore, the role of radiation can often be neglected. 

We can absorb the ambient temperature into the input vector and the linear convection term into the state vector. Now, the governing equations become:

$$\begin{aligned}
\begin{bmatrix} \dot{\tau}_1 \\ \vdots \\ \dot{\tau}_3 \end{bmatrix} = \begin{bmatrix} \frac{-k_{12} - h_a a_1}{m_1 c_{p,1}} & \frac{k_{12}}{m_1 c_{p,1}} & 0 \\
\frac{k_{12}}{m_2 c_{p,2}} & \frac{-k_{12} - k_{23} - h_a a_2}{m_2 c_{p,2}} & \frac{k_{23}}{m_2 c_{p,2}} \\
 0 & \frac{k_{23}}{m_3 c_{p,3}} & \frac{-k_{23} - h_a a_3}{m_3 c_{p,3}} \end{bmatrix} \begin{bmatrix} \tau_1 \\ \vdots \\ \tau_3 \end{bmatrix} + \begin{bmatrix} r(\tau_1, \tau_a, 1, t) \\ r(\tau_2, \tau_a, 2, t) \\ r(\tau_3, \tau_a, 3, t) \end{bmatrix} + \begin{bmatrix} \frac{h_a a_1}{m_1 c_{p,1}} & \frac{1}{m_1 c_{p,1}} & 0 & 0 \\ \frac{h_a a_2}{m_2 c_{p,2}} & 0 & \frac{1}{m_2 c_{p,2}} & 0 \\ \frac{h_a a_3}{m_3 c_{p,3}} & 0 & 0 & \frac{1}{m_3 c_{p,3}} \end{bmatrix} \begin{bmatrix} \tau_a \\ u_1 \\ u_2 \\ u_3 \end{bmatrix}  \, .
\end{aligned}$$

In these equations, we can distinguish three types of quantities:
1. Measured/observed quantities: e.g. $\tau_a$, $\tau_i$, $u_i$. These may vary over time, and are known up to a given accuracy due to measurement noise;

2. Known constants: e.g. $m_i$, $c_{p, i}$, $a$. These are fully known, and are constant over time. This is reasonable for quantities such as mass $m$ and surface area $a$ (which can be easily measured) and specific heat capacity $c_p$;

3. Unknown constants: e.g. $k_{ij}$, $h_a$. Yhere is no simple physical way to measure or derive their values. For example, the conduction coefficients $k_{ij}$ can vary depending on how tightly the blocks have been put together. In this project, we want to identify these constants using Bayesian inference.

In [ ]:
using Revise
using RxInfer
using Random
using MAT
using Optim
using ControlSystems
using Polynomials
using BlockDiagonals
using DifferentialEquations
using Distributions
using LinearAlgebra
using Plots; default(label="", linewidth=3, margin=15Plots.pt)

include("../../util/visualizations.jl")
include("../../util/util.jl")

## Load data

In [ ]:
dat = matread("data\\2023\\data\\2julia.mat")

In [ ]:
# Sensors per block
Ns = convert.(Int64, dat["N"])
N = sum(Ns)

# Sensor 7 is broken
NTCIX = [1:6; 8:13];
measurements = hcat([dat["NTC$k"] for k in NTCIX]...)
inputs = dat["u"]; # First column is ambient

In [ ]:
# Time parameters
Ts = dat["Ts"]
tmax = convert(Int64, dat["tmax"]);

# Subsample
ssix = 1
Δt = Ts*ssix
time = range(0.0, step=Δt, stop=tmax-1)
T = length(time);

In [ ]:
inputs_ss = inputs[1:ssix:tmax,:]
measurements_ss = measurements[1:ssix:tmax,:];

In [ ]:
lcolors = hcat([repeat(["red"], Ns[1]); repeat(["blue"], Ns[2]); repeat(["orange"], Ns[3])]...)
labelsm = hcat(["τ_$k measured" for k in NTCIX]...)

vis_ss = 1_000

plt1 = plot(time[2:vis_ss:end], 
            measurements_ss[2:vis_ss:end,:],
            linecolors = lcolors, 
            labels = labelsm,
#             xscale=:log10,
            legend=:topleft,
            xlabel="time (s)",
            ylabel="temp (C)")
plt2 = plot(time[2:vis_ss:end], 
inputs_ss[2:vis_ss:end, 2:end],
            ylabel="input heat (W)")
plot(plt1,plt2, layout=(2,1), size=(900,800))

In [ ]:
mcps = dat["Mcps"]         # Mass heat capacity
gc   = dat["gc"]           # Base conduction of blocks
Ac   = dat["Ac"]           # Cross-section surfaces (m2)
An   = dat["An"]           # Outer surface areas (m2)
l    = dat["l"]            # Lengths
lpl  = dat["l_pom_left"]
lpr  = dat["l_pom_right"]

An_ = vcat([repeat([An[k]], Ns[k]) for k in 1:3]...)
M   = diagm(vcat([repeat([mcps[k]], Ns[k]) for k in 1:3]...))

## Maximum likelihood of conductance parameters and linear convection


Let $x = [\tau_1 \ \tau_2 \ \tau_3]$ and $u = [\tau_a \ u_1 \ \dots \ u_3]$. If we ignore the nonlinear convection term, we have:

$$\begin{aligned}
\begin{bmatrix} \dot{\tau}_1 \\ \vdots \\ \dot{\tau}_3 \end{bmatrix} = \underbrace{\begin{bmatrix} \frac{-k_{12} - h_a a_1}{m_1 c_{p,1}} & \frac{k_{12}}{m_1 c_{p,1}} & 0 \\
\frac{k_{12}}{m_2 c_{p,2}} & \frac{-k_{12} - k_{23} - h_a a_2}{m_2 c_{p,2}} & \frac{k_{23}}{m_2 c_{p,2}} \\
 0 & \frac{k_{23}}{m_3 c_{p,3}} & \frac{-k_{23} - h_a a_3}{m_3 c_{p,3}} \end{bmatrix}}_{F} \begin{bmatrix} \tau_1 \\ \vdots \\ \tau_3 \end{bmatrix} + \underbrace{\begin{bmatrix} \frac{h_a a_1}{m_1 c_{p,1}} & \frac{1}{m_1 c_{p,1}} & 0 & 0 \\ \frac{h_a a_2}{m_2 c_{p,2}} & 0 & \frac{1}{m_2 c_{p,2}} & 0 \\ \frac{h_a a_3}{m_3 c_{p,3}} & 0 & 0 & \frac{1}{m_3 c_{p,3}} \end{bmatrix}}_{G} \begin{bmatrix} \tau_a \\ u_1 \\ u_2 \\ u_3 \end{bmatrix}  \, .
\end{aligned}$$

We discretize this using a forward Euler procedure and inject a tiny bit of zero-mean Gaussian noise:

$$\begin{aligned}
x_{k+1} = A x_k + B u_k + w_k\, , \quad \text{with}\ w_k \sim \mathcal{N}(0, Q) \, ,
\end{aligned}$$

where

$$\begin{aligned}
    A = (I + \Delta t F \big) \, , \quad B = \Delta t G \, , \quad Q = \epsilon I \, .
\end{aligned}$$

Finally, we have noisy measurements of temperatures:
$$\begin{aligned}
y_{k} = x_k + v_k\, , \quad \text{with}\ v_k \sim \mathcal{N}(0, R) \, .
\end{aligned}$$

In [ ]:
@model function SSM(A, B, C, Q, R, m0, S0; T=1)
    
    x = randomvar(T)
    u = datavar(Vector{Float64}, T)
    y = datavar(Vector{Float64}, T)
    
    x_0 ~ MvNormalMeanCovariance(m0, S0)
    x_kmin1 = x_0
    for k = 1:T
        
        x[k] ~ MvNormalMeanCovariance(A*x_kmin1 + B*u[k], Q)
        y[k] ~ MvNormalMeanCovariance(C*x[k], R)
        
        x_kmin1 = x[k]
    end
end

In [ ]:
Dx = 12
Du = 13
Dy = 12
C    = diagm(ones(Dy))
Q    = 1e-4*diagm(ones(Dx))
R    = diagm(1e-6*ones(Dy))        
S0   = diagm(1e-6*ones(Dx))
m0   = measurements_ss[1,:];

In [ ]:
output_ = [measurements_ss[k,:] for k in 1:T];
inputs_ = [inputs_ss[k,:] for k in 1:T];

In [ ]:
function J(θ::AbstractVector)
    "θ = [k_12, k_23, h_a]"
    
    K = conductances(θ, Ns, An, gc, [lpl, lpr])
    A = inv(M)*K
    
    B = Δt*inv(M)*[θ[3]*An_ diagm(ones(N))]
    
    results = inference(
        model       = SSM(A, B, C, Q, R, m0,S0, T=T),
        data        = (y = output_, u = inputs_),
        options     = (limit_stack_depth = 100,),
        free_energy = true
    )      
    return results.free_energy[end]    
end

In [ ]:
# ops = Optim.Options(g_tol=1e-8, time_limit=30.0, show_every=1)
# res = optimize(J, 0.0, 10.0, 3*ones(3), Fminbox(LBFGS()), ops; autodiff=:forward)

In [ ]:
# k_12,k_23,h_a = Optim.minimizer(res)
k_12, k_23, h_a = dat["xoptimal"]

In [ ]:
K = conductances([k_12,k_12,h_a], Ns, An, gc, [lpl, lpr])
A = exp(Δt*inv(M)*K)
B = Δt*inv(M)*[h_a*An_ diagm(ones(N))]
    
results = inference(
    model       = SSM(A, B, C, Q, R, m0, S0, T=T),
    data        = (y = output_, u = inputs_),
    options     = (limit_stack_depth = 100,),
)

In [ ]:
qx = results.posteriors[:x]
fitx_v = hcat( var.(qx)...)
fitx_m = hcat(mean.(qx)...)

In [ ]:
labelsf = hcat(["τ_$k fit" for k in NTCIX]...)

plot(time[2:vis_ss:end],
     fitx_m[:,2:vis_ss:end]';
     ribbon=sqrt.(fitx_v[:,2:vis_ss:end])',
     legend = :topleft, 
     linecolors = lcolors, 
     labels = labelsf,
     xlabel = "time [s]", 
     ylabel = "temperature [C]",
     size=(900,400)
)
plot!(time[2:vis_ss:end], 
      measurements_ss[2:vis_ss:end,:], 
      alpha = 0.3,
      linestyle = :dash,
      linecolors = lcolors, 
#       labels = labelsm,
#       xscale=:log10,
)

In [ ]:
savefig("figures/validation-ML-SSM-states.png")

## Simulation

In [ ]:
# sys = ss(A,B,C,zeros(Dx,Du))
# sims = lsim(sys, inputs_ss', time, x0=dat["T0"]', method=:zoh);

In [ ]:
sims  = zeros(T,Dx)
sims[1,:] = dat["T0"]
for ii in 2:T
    sims[ii,:] = A*sims[ii-1,:] + B*inputs_ss[ii,:]
end

In [ ]:
SIM_MSE = mean( (sims - measurements_ss).^2 )

In [ ]:
labelss = hcat(["τ_$k simulated" for k in NTCIX]...)

plot(time[2:vis_ss:end],
     sims[2:vis_ss:end,:],
     legend = true, 
     linecolors = lcolors, 
     labels = labelss,
     xlabel = "time [s]", 
     ylabel = "temperature [C]",
     size=(900,400),
     title="MSE = $SIM_MSE",
)
plot!(time[2:vis_ss:end], 
      measurements_ss[2:vis_ss:end,:], 
      alpha = 0.5,
      linecolors = lcolors, 
      linestyle = :dash,
)

In [ ]:
savefig("figures/validation_ML-SSM-simulations.png")

In [ ]:
matwrite("results/ML-SSM.mat", 
    Dict("A" => A,
         "B" => B,
         "Q" => Q,
         "y_sim" => sims[1],
         "Ts" => Δt);
)